# Creating a Top Movies DataFrame

## Introduction 

I am going to create a DataFrame of the top 50 pages of Movies ordered by average voted score listed using an API from TMDB. I will also then clean the DataFrame, checking for duplicates, appropriate data types, and null values before saving it as a csv file ready for analysis. 

My aim is to create repeatable code that can save a DataFrame to analyse at a later date, doing this also allows me to conduct analysis seperate to the data collection, and also allows me to conduct multiple rounds of data collection and analyse differences in voted score over time.
 
### Importing libraires that we will be using

In [1]:
from datetime import datetime
import pandas as pd
import requests
import json
import time

 ## Setting up the API
 
 After going onto the TMDB website and signed up to gain an authorization code to be able to use the API, I started off with the basic API request script provided for the Top Movies list endpoint. I modified the query parameters to sort the movies by the average score voted (giving us the top movies by *vote_average*), where the average vote was greater than 7.5, and there was at least 300 votes that made up the score. I also chose to exclude *documentary* and *TV Movie* genres.
 
 When running the query against the API endpoint, we can see that it returns 20 movies per page. I created a `for` loop to get all the top 50 pages of movies. Within the `for` loop, we convert the json response into a dictionary. We then insert each movie from the response into a `movies` dictionary using the *tmdb_id* as the key. By doing this, we are able to deduplicate the data we receive. It is possible for this to happen if ratings change in between our requests to the API. After the data collection, we turn the `movies` dictionary into a list for easier consumption.
 
 As this is a lot to ask the TMDB server to send us, we have added a sleep query of 0.025 seconds between each request since they have stated on their [website](https://developer.themoviedb.org/docs/rate-limiting) that they accept up to around 50 requests per second with this subject to change. Therefore our 0.025 second break means that we are requesting up to a maximum of 40 requests per second. Getting 50 pages could take a while, so on each loop I print which page it has requested so we have a visual track of how the request is going. 

In [2]:
movies = {}

with open('access_code.json') as f:
    data = json.load(f)
    access = data['authorisation']

for i in range(1, 51):
    url = f"https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page={i}&sort_by=vote_average.desc&vote_average.gte=7.5&vote_count.gte=300&without_genres=99%2C10770"
    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {access}"
    }
    response = requests.get(url, headers=headers)
    page = json.loads(response.text)
    for movie in page['results']:
        movies[movie['id']] = movie
    print(i)
    time.sleep(0.025)

movies = list(movies.values())

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [3]:
print(movies[:3])

[{'adult': False, 'backdrop_path': '/kXfqcdQKsToO0OUXHcrrNCHDBzO.jpg', 'genre_ids': [18, 80], 'id': 278, 'original_language': 'en', 'original_title': 'The Shawshank Redemption', 'overview': 'Framed in the 1940s for the double murder of his wife and her lover, upstanding banker Andy Dufresne begins a new life at the Shawshank prison, where he puts his accounting skills to work for an amoral warden. During his long stretch in prison, Dufresne comes to be admired by the other inmates -- including an older prisoner named Red -- for his integrity and unquenchable sense of hope.', 'popularity': 162.529, 'poster_path': '/9cqNxx0GxF0bflZmeSMuL5tnGzr.jpg', 'release_date': '1994-09-23', 'title': 'The Shawshank Redemption', 'video': False, 'vote_average': 8.704, 'vote_count': 25966}, {'adult': False, 'backdrop_path': '/tmU7GeKVybMWFButWEGl2M4GeiP.jpg', 'genre_ids': [18, 80], 'id': 238, 'original_language': 'en', 'original_title': 'The Godfather', 'overview': 'Spanning the years 1945 to 1955, a ch

## Understanding the Movie Genre

Now we have our *Top Movies*, after printing the first three movies in our list, we can see we are given *genre_ids*. So we need to get another API to translate these ids into the names of the genres. 

To fetch this data, we can use the `/genre/movie/list` endpoint. Here we are querying that endpoint that lists all movie genres and we create a list of dictionaries.

In [1]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {access}"
}

response = requests.get(url, headers=headers)
genres = list(json.loads(response.text).values())[0]
print(genres)

[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 99, 'name': 'Documentary'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 14, 'name': 'Fantasy'}, {'id': 36, 'name': 'History'}, {'id': 27, 'name': 'Horror'}, {'id': 10402, 'name': 'Music'}, {'id': 9648, 'name': 'Mystery'}, {'id': 10749, 'name': 'Romance'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 10770, 'name': 'TV Movie'}, {'id': 53, 'name': 'Thriller'}, {'id': 10752, 'name': 'War'}, {'id': 37, 'name': 'Western'}]


## Creating the DataFrame

Now we have our information for our top movies, we create our DataFrame. We begin by making lists of the information that we would like in our DataFrame from the dictionaries we have obtained. With analysis I may want to do in mind, I am taking the *id* of the movie, the movie *release_date*, the *vote_average*, movie *title*, *genre_ids*, movie *popularity* score and *vote_count*. 

I have created another for loop using the list of genres to rename all the *genre_ids* into the name of the genres for readibility. Finally, we can create the DataFrame with all the information from the lists we have obtained ensuring that python understands that release dates are dates using our datetime library. Then we have also sorted our movies by their release date.

In [5]:
ids = []
dates = []
score = []
title = []
genre = []
popularity = []
vote = []

#creating lists of information from movie dictionaries
for movie in movies:
    ids.append(movie['id'])
    dates.append(movie['release_date'])
    score.append(movie['vote_average'])
    title.append(movie['title'])
    genre.append(movie['genre_ids'])
    popularity.append(movie['popularity'])
    vote.append(movie['vote_count'])

#converting genre id into the genre name
for dic in genres:
    for x, lis in enumerate(genre):
        for i in range(0,len(lis)):
            if lis[i] == dic['id']:
                genre[x][i] = dic['name']
            
#creating the dataframe, ensuring dates are date type and movies in date order
top_movies = pd.DataFrame(data={'tmdb_id':ids, 'movie_title':title, 'release_date':dates, 'vote_count':vote, 'average_score':score, 'popularity_score':popularity, 'genres':genre})
top_movies['release_date'] = pd.to_datetime(top_movies['release_date'])
top_movies = top_movies.sort_values(by='release_date').reset_index(drop=True)
top_movies

,tmdb_id,movie_title,release_date,vote_count,average_score,popularity_score,genres
0,775,A Trip to the Moon,1902-04-17,1687,7.931,20.315,"[Adventure, Science Fiction]"
1,234,The Cabinet of Dr. Caligari,1920-02-27,1480,7.946,18.525,"[Drama, Horror, Thriller, Crime]"
2,10098,The Kid,1921-01-21,1980,8.184,18.306,"[Comedy, Drama]"
3,653,Nosferatu,1922-02-16,1867,7.703,29.647,[Horror]
4,22596,Safety Last!,1923-04-01,465,7.762,8.897,"[Comedy, Romance]"
...,...,...,...,...,...,...,...
995,976893,Perfect Days,2023-11-10,642,7.900,99.017,[Drama]
996,792307,Poor Things,2023-12-07,3026,7.811,435.971,"[Science Fiction, Romance, Comedy]"
997,906126,Society of the Snow,2023-12-15,2429,8.069,88.672,"[Drama, History]"
998,850165,The Iron Claw,2023-12-21,423,7.600,75.562,"[Drama, History]"


## Understanding our DataFrame

Before proceeding, it's important to ensure we understand what each column of the DataFrame represents;

* **tmdb_id** - *The movie id on the TMDB website, this will be useful should we want to obtain more information about the movie from the website API*
* **movie_title** - *The title of the movie*
* **release_date** - *The date that the movie was released*
* **vote_count** - *The number of votes that make up the average score of the film*
* **average_score** - *The average score given by voters out of 10*
* **popularity_score** - *The score that TMDB has given the film based on votes, views, number of users adding the movie to their 'favourite' or 'watchlist', total votes, release date and previous days score* ([source](https://developer.themoviedb.org/docs/popularity-and-trending#popularity))
* **genres** - *The the genres that the film represents*

## Cleaning our DataFrame

Now we have a DataFrame, we can begin to ensure that we happy with the values we have before we save it.

We have up to around 1,000 movies with no null values. We can also check that movies are not duplicated on multiple pages, we want to check both movie *tmdb_id* and *movie_title*, however, we can see that where there are duplicates in the titles, these films have different release dates and the *movie_id* are also different, meaning they should be different movies. In contradiction, when running this, I have seen Whiplash from 2013 and 2014, the 2013 movie is a short film. Also, Miracle in Cell No7 from 2013 and 2019, the 2013 version is the original Korean version and the 2019 versian is an Indonesian adaption. When we begin analysis, it would be important to manually review these duplicates and finding further details about the movies such as runtimes and original language to decide whether we want to include these movies.

We are also checking that the format of each column is correct by checking the *dtypes*. We can see that the *tmdb_id* and *vote_count* are whole numbers, *average_score* and *popularity_score* are numbers, *movie_title* and *genres* are objects and *release_date* are datetimes. These are as we want them. 

In [6]:
top_movies.tmdb_id.nunique()

1000

In [7]:
top_movies.isnull().sum()

tmdb_id             0
movie_title         0
release_date        0
vote_count          0
average_score       0
popularity_score    0
genres              0
dtype: int64

In [8]:
uniquelist = []
duplicatelist = []
for i in top_movies.tmdb_id:
    if i not in uniquelist:
        uniquelist.append(i)
    elif i not in duplicatelist:
        duplicatelist.append(i)
        
print(duplicatelist)

[]


In [9]:
uniquelist = []
duplicatelist = []
for i in top_movies.movie_title:
    if i not in uniquelist:
        uniquelist.append(i)
    elif i not in duplicatelist:
        duplicatelist.append(i)
for x, i in enumerate(top_movies.movie_title):
    if i in duplicatelist:
        print(top_movies.iloc[x]['tmdb_id'],":", i, ":", top_movies.iloc[x]['release_date'].strftime("%m/%d/%Y"))
print(duplicatelist)

143 : All Quiet on the Western Front : 04/29/1930
10835 : The Killer : 03/24/1989
367412 : Whiplash : 01/18/2013
158445 : Miracle in Cell No. 7 : 01/23/2013
244786 : Whiplash : 10/10/2014
637920 : Miracle in Cell No. 7 : 10/10/2019
938008 : The Killer : 07/13/2022
49046 : All Quiet on the Western Front : 10/07/2022
['Whiplash', 'Miracle in Cell No. 7', 'The Killer', 'All Quiet on the Western Front']


In [10]:
top_movies.dtypes

tmdb_id                      int64
movie_title                 object
release_date        datetime64[ns]
vote_count                   int64
average_score              float64
popularity_score           float64
genres                      object
dtype: object

##  Saving the DataFrame 

Finally, I will save our DataFrame as csv file ready for analysis. Saving the DataFrame as csv before analysis means that when we re-run the code, the movies and scores that we have won't change, therefore any analysis we complete will stay relevant to the data that we have. We can always re-run the API and get the latest list of Top Movies to analyse and complete new analysis on the most current findings.  


In [11]:
top_movies.to_csv('Top_TMDB_Movies.csv', index=False)

## Conclusion

We have created our DataFrame ready for analysis using an API request from the TMDB website. We have reviewed the data that there are no null values and the data is in the correct format ready for working with.

### Data Source

Movie data and API from:
https://www.themoviedb.org/?language=en-GB 

API request limiting:
https://developer.themoviedb.org/docs/rate-limiting